# SQL 2 

## Agregacion

¿Que hace esta consulta?

```sql
SELECT AVG(precio) 
FROM productos 
WHERE fabricante = 'Toyota';
```
Entrega el precio **promedio** de los productos fabricados por Toyota.

Podemos usar:

- `AVG` para obtener el promedio de una columna.
- `SUM` para obtener la suma de una columna.
- `COUNT` para obtener la cantidad de filas.
- `MAX` para obtener el valor máximo de una columna.
etc.

## Caso especial de `COUNT`

```sql
SELECT COUNT(*)
FROM productos
WHERE año > 2012;
```

Entrega la cantidad de productos que fueron fabricados después del 2012. 

COUNT(*) cuenta los **nulls**

```sql
SELECT COUNT(fabricante)
FROM productos
WHERE año > 2012;
```

Entrega la cantidad de productos que fueron fabricados después del 2012 y que tienen un fabricante distinto de `null`.

OJO: EN ambnos se cuerntan los **duplicados**.

## Group BY

```sql
SELECT fabricante, COUNT(fabricante)
FROM Productos
WHERE año > 2012
GROUP BY fabricante;
```

Esta consulta:
* Computa los resultados según el FROM y WHERE
* Agrupa los resultados según los atributos del GROUP BY
* Para cada grupo se aplica independientemente la
agregación

## HAVING

Misma consulta, pero sólo queremos los productos
que se vendieron más de 100 veces

```sql
SELECT producto, SUM(precio*cantidad) AS ventaTotal
FROM Compra
WHERE fecha > ’10/01’
GROUP BY producto
HAVING SUM(cantidad) > 100
```

## Consultas con Agregación

```sql
SELECT <S>
FROM R1, …, Rn
WHERE <Condición 1>
GROUP BY a1, …, ak
HAVING <Condición 2>
```
* S puede contener atributos de a1, …, ak y/o agregados,
pero ningún otro atributo
* Condición 1 es una condición que usa atributos de R1,
…, Rn
* Condición 2 es una condición de agregación de los
atributos de R1, …, Rn

* Se computa el FROM - WHERE de R1, …, Rn
* Agrupar la tabla por los atributos de a1, …, ak
* Computar los agregados de la Condición 2 y mantener grupos que satisfacen
* Computar agregados de S y entregar el resultado


## Consultas Anidadas

Considerando el esquema:

Bandas(nombre, vocalista, ...)
Estudiantes_UC(nombre, ...)
Toco_en(nombre_banda, nombre_festival)

Obtengamos todas las bandas cuyo vocalista sea un
estudiante UC y que hayan tocado en Lollapallooza

```sql
SELECT Bandas.nombre
FROM Bandas, Estudiantes_UC
WHERE Bandas.vocalista = Estudiantes_UC.nombre
AND Bandas.nombre IN (
SELECT Toco_en.nombre_banda
FROM Toco_en
WHERE Toco_en.nombre_festival = ‘Lollapalooza’
);
```

EN la parte del in: Comprobamos que Bandas.nombre esté dentro
del listado de bandas que han tocado en
Lollapalooza.

Entonces hacemos 2 consultas en 1, la primera es la que está dentro del IN y la segunda es la que está fuera.

Donde la primera se encarga de obtener las bandas que han tocado en Lollapalooza y la segunda se encarga de obtener las bandas cuyo vocalista sea un estudiante UC.

Al unirlas con el IN, obtenemos las bandas que cumplen con ambas condiciones.

Si la sub consulta retorna un escalar podemos
usar las operadores condicionales típicos. Si no
podemos hacerlo agregando un operador
adicional.


## All, ANY

Cervezas(nombre, precio, ...)

Cervezas más baratas que la Austral Calafate

```sql
SELECT Cervezas.nombre
FROM Cervezas
WHERE Cervezas.precio < ALL (
SELECT Cervezas2.precio
FROM Cervezas AS Cervezas2
WHERE Cervezas2.nombre = ‘Austral Calafate’
);
```

Cervezas que no son la más cara

```sql
SELECT Cervezas.nombre
FROM Cervezas
WHERE Cervezas.precio < ANY (
SELECT Cervezas2.precio
FROM Cervezas AS Cervezas2
);
```


## Podemos expresar (ALL y ANY) con SFW (Select From Where)

LAs consultas SFW son **monotonas**. Si se agrega una fila a la tabla, el resultado no cambia.

ALL **no es monótono**. Si se agrega una fila a la tabla, el resultado puede cambiar.

ANY si es monótono.

Por lo tanto, ALL no se puede expresar con SFW, pero ANY si.

## SUb consultas relacionadas

Nombres de películas que se repiten en años
diferentes

```sql
SELECT p.nombre
FROM Películas AS p
WHERE p.año <> ANY (
SELECT año
FROM Películas
WHERE nombre = p.nombre
);
```
La sub consulta depende de la externa!



## COnsultas Anidadas COMO joins

El nombre de cada actor junto con el total de películas en las
que ha actuado

```sql
SELECT Actores.nombre, agg.cuenta
FROM Actores, (
SELECT id_actor as id, COUNT(*) as cuenta
FROM Actuo_en
GROUP BY id_actor
) as agg
WHERE Actores.id = agg.id;
```

El nombre de cada actor junto con el año de la primera película
en la que actuó.

```sql
SELECT Actores.nombre, agg.año
FROM Actores, (
SELECT Actuo_en.id_actor as id, MIN(Peliculas.año) as año
FROM Actuo_en, Peliculas
WHERE Actuo_en.id_pelicula = Peliculas.id
GROUP BY id_actor
) as agg
WHERE Actores.id = agg.id;
```
¿Qué pasa si queremos el nombre de la película además del año?

```sql
SELECT Actores.nombre, agg.año, Peliculas.nombre
FROM Actores, (
SELECT Actuo_en.id_actor as id, MIN(Peliculas.año) as año, Peliculas.nombre as nombre
FROM Actuo_en, Peliculas
WHERE Actuo_en.id_pelicula = Peliculas.id
GROUP BY id_actor
) as agg
WHERE Actores.id = agg.id;
```





## Información incompleta

### NUlos

Pueden signifircar:
* El valor el exitse pero no tengo la info
* Valo no existe (si nombre ? null la ionfo no existe)
* No se si el valor existe o no

La consulta: 

```sql
SELECT * FROM R
```	

Equivale a la union de:

```sql
SELECT * FROM R WHERE R.b = 3
SELECT * FROM R WHERE R.b <> 3
SELECT * FROM R WHERE R.b IS NULL
```

PAra ver si un elemento es nulo usamos `IS NULL`, para ver si no es nulo usamos `IS NOT NULL`

**Si un valor de una columna es nulo, cualquier operación que involucre ese valor resultará en nulo.**

Para la lofgica usa tres valores SQL:
* TRUE
* FALSE
* UNKNOWN

AL reqaliazar igualdades con nulos, el resultado es UNKNOWN.

## Agregacion de Nulos

La suma de nulos es nulo, el promedio de nulos es nulo, etc.

Al busvcart la suma de un atributo cion (1, null)
la suma es 1, ya que el null se ignora.

## REstriccion denulos al crear una tabal en una columna

```sql
CREATE TABLE R (
a INT NOT NULL,
b INT
);
```



## Inner Joins en SQL

Normalmentre haciamos joins con FROM y WHERE

```sql
SELECT *
FROM Peliculas, Actuo_en
WHERE id = id_pelicula
```

Pero esto es equivalente a:

```sql
SELECT *
FROM Peliculas INNER JOIN Actuo_en
ON id = id_pelicula
```

O tambien a:

```sql
SELECT *
FROM Peliculas JOIN Actuo_en
ON id = id_pelicula
```


## Outer joins en SQL

Escribamos una consulta que liste los ingresos totales
de cada estudio.

Creamos primero ambas tablas, estudios y peliculas

```sql
CREATE TABLE Estudios (
    nombre VARCHAR(255),
    titulo VARCHAR(255),
);

CREATE TABLE Peliculas (
    titulo VARCHAR(255),
    ingreso INT
);
```

Luego hacemos la consulta:

```sql
SELECT Estudios.nombre, SUM(Peliculas.ingreso)
FROM Estudios JOIN Peliculas
ON Estudios.titulo = Peliculas.titulo
GROUP BY Estudios.nombre;
```

Aqui el **problema es que si un estudio no aparece en la tabla de peliculas, no aparecerá en el resultado**.

Lo solucionamos con un **outer join izquierdo**:

```sql
SELECT Estudios.nombre, SUM(Peliculas.ingreso)
FROM Estudios LEFT OUTER JOIN Peliculas
ON Estudios.titulo = Peliculas.titulo
GROUP BY Estudios.nombre;
```

Esto hara que: Si un estudio no aparece en la tabla de peliculas, aparecerá en el resultado con un ingreso de **null**.

## Resumen OUTER JOINS.

* R LEFT OUTER JOIN S: mantenemos las tuplas de
R que no tienen correspondencia.
* R RIGHT OUTER JOIN S: mantenemos las tuplas
de S que no tienen correspondencia.
* R FULL OUTER JOIN S: mantenemos las tuplas de
R y S que no tienen correspondencia


![resuemen_outer_joins](img/outer_join.png)

## ACotrar consultas

En una consulta anterior:

```sql
SELECT Bandas.nombre
FROM Bandas, Estudiantes_UC
WHERE Bandas.vocalista = Estudiantes_UC.nombre
AND Bandas.nombre IN (
SELECT Toco_en.nombre_banda
FROM Toco_en
WHERE Toco_en.nombre_festival = ‘Lollapalooza’
);
```

Esto es equialente a:

```sql
SELECT DISTINCT Bandas.nombre
FROM Bandas, Estudiantes_UC, Toco_en
WHERE Bandas.vocalista = Estudiantes_UC.nombre
AND Banda.nombre = Toco_en.nombre_banda
AND Toco_en.nombre_festival = ‘Lollapalooza’;
```

Utilizamos DISTINCT en este caso para evitar duplicados, y es equivalente a usar `IN` en la subconsulta. Dado que `IN` no permite duplicados.
